# Appartment search 


In this notebook we'll provide an example for using different openrouteservice API's to help you look for an apartment.

In [1]:
mkdir ors-apartment
conda create -n ors-apartment python=3.6 shapely
cd ors-apartment
pip install openrouteservice ortools folium

SyntaxError: invalid syntax (<ipython-input-1-452fd2304efa>, line 1)

In [1]:
import folium
import json
import os
import matplotlib as mpl
from openrouteservice import client

We have just moved to San Francisco with our kids and are looking for the perfect location to get a new home. Our geo intuition tells us we have to look at the data to come to this important decision. So we decide to geek it up a bit.

For this analysis, we will again use the Open Router Serive API. Therefore, we need to import the necessary modules and create a client with our personal API key.

In [2]:
api_key = '5b3ce3597851110001cf62483ae5cf98ae4743649600ff877943b339' #Provide your personal API key
clnt = client.Client(key=api_key) 

# Apartment isochrones

There are three different suggested locations for our new home. Let's visualize them and the 15 minute walking radius on a map:

First, we store the locations of the three potential appartments in a dictionary.

In [3]:
# Set up the apartment dictionary with real coordinates
appartments = {'first_appartment': {'location': [-122.430954, 37.792965]},
            'second_appartment': {'location': [-122.501636, 37.748653]},
            'third_appartment': {'location': [-122.446629, 37.736928]}
           }

Then, we will request the isochrones around each apparment and visualize them in the map. 

In [7]:
# Settings for isochrones
profile = 'foot-walking'
intervals = [1000] # in seconds (if range_type=time) or meters (if range_type=distance)
range_type = 'time'
export_isochrones = True
path_out = '/Users/chludwig/Data/temp/'

#Put together the parameters of the ORS request of isochrones 
params_iso = {'profile': profile,
              'intervals': intervals, # 900/60 = 15 mins
              'segments': intervals[0],
              'range_type': range_type,
              'attributes': ['area'] # Get area of each isochrone
             }

style_function = lambda x: {'weight': 1}

# Set up folium map
mapSF = folium.Map(tiles='Stamen Toner', location=([37.738684, -122.450523]), zoom_start=12)
# Display map
mapSF

# Iterate over all appartments and request isochrones for each one and add them to the map
for name, apt in appartments.items():
    # Add apartment coords to request parameters
    params_iso['locations'] = apt['location'] 
    # Perform isochrone request
    apt['iso'] = clnt.isochrones(**params_iso) 
    # Add resulting isochrones to the map
    folium.features.GeoJson(apt['iso'], style_function=style_function, name='Isochrones of ' + name).add_to(mapSF) 
    # Add icon at the location of the appartment
    folium.map.Marker(list(reversed(apt['location'])), # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                        icon_color='#cc0000',
                                        icon='home',
                                        prefix='fa',
                                       ),
                      popup=name,
                 ).add_to(mapSF)
    # Export to file 
    if export_isochrones:
        with open(os.path.join(path_out, 'isochrones_' + name +'.geojson'), "w") as fp: 
            json.dump(apt['iso'], fp)

mapSF.add_child(folium.LayerControl())
mapSF

# Count POIs within isochrones around the appartments


For the ever-styled foodie parents we are, we need to have the 3 basic things covered: kindergarten, supermarket and hair dresser. Let's see what options we got around our apartments:

In [8]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance'}

# POI categories according to 
# https://github.com/GIScience/openrouteservice-docs#places-response
categories_poi = {'kindergarten': [153],
                  'supermarket': [518],
                  'hairdresser': [395]}

# Iterate over appartments and count the number of POIs within the FIRST isochrone
for name, apt in appartments.items():
    apt['categories'] = dict() # Store in pois dict for easier retrieval
    params_poi['geojson'] = apt['iso']['features'][0]['geometry']
    print('Number of POIs around {}'.format(name))
    
    # Iterate over POI categories
    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apt['categories'][typ] = dict()
        apt['categories'][typ]['geojson']= clnt.places(**params_poi)['features'] # Actual POI request
        print("\t{}: {}".format(typ, # Print amount POIs
                                len(apt['categories'][typ]['geojson'])))

Number of POIs around first_appartment
	kindergarten: 1
	supermarket: 12
	hairdresser: 12
Number of POIs around second_appartment
	kindergarten: 3
	supermarket: 1
	hairdresser: 5
Number of POIs around third_appartment
	kindergarten: 1
	supermarket: 3
	hairdresser: 2


So, all apartments meet all requirements. Seems like we have to drill down further.

# Calculate routes from apartments to nearby POIs

To decide on a place, we would like to know from which apartment we can reach all required POI categories the quickest. So, first we look at the distances from each apartment to the respective POIs.

In [9]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
               'format_out': 'geojson',
               'geometry': 'true',
               'geometry_format': 'geojson',
               'instructions': 'false',
               }

# Set up dict for font-awesome
style_dict = {'kindergarten': 'child',
              'supermarket': 'shopping-cart',
              'hairdresser': 'scissors'
             }

# Store all routes from all apartments to POIs
for apt in appartments.values():
    for cat, pois in apt['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']
            
            # Perform actual request
            params_route['coordinates'] = [apt['location'],
                                           poi_coords
                                          ]
            json_route = clnt.directions(**params_route)
            
            folium.features.GeoJson(json_route).add_to(mapSF)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                              )
                             ).add_to(mapSF)
            
            poi_duration = json_route['features'][0]['properties']['summary'][0]['duration']
            pois['durations'].append(poi_duration) # Record durations of routes
        
mapSF

_OverQueryLimit: 429 ({'error': 'Rate limit exceeded'})

# Quickest route to all POIs

Now, we only need to determine which apartment is closest to all POI categories.

In [6]:
# Sum up the closest POIs to each apartment
for name, apt in apt_dict.items():
    apt['shortest_sum'] = sum([min(cat['durations']) for cat in apt['categories'].values()])
    print("{} apartments: {} mins".format(name,
                                          apt['shortest_sum']/60
                                         )
         )

first apartments: 37.098333333333336 mins
second apartments: 40.325 mins
third apartments: 35.315000000000005 mins


# We got a winner!

Finally, it looks like the 3rd apartment is the one where we would need to walk the shortest amount of time to reach a kindergarten, supermarket and a hair dresser. Let's pack those boxes and welcome to San Francisco.